In [18]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()
import MathOptInterface as MOI
import Ipopt 
import FiniteDiff
import ForwardDiff as FD
import Convex as cvx 
import ECOS
using LinearAlgebra
using Plots; plotly()
using Random
using JLD2
using Test
using MeshCat
const mc = MeshCat
# using TrajOptPlots
# using StaticArrays
using Printf

  Activating environment at `~/Dropbox/My Mac (MacBook Pro (2))/Desktop/CMU/Optimal Control/Optimal-Controls-Gymnastics-Bot/MonkeyBarBot/Project.toml`


In [60]:
include(joinpath(@__DIR__, "utils","fmincon.jl"))
include(joinpath(@__DIR__, "utils", "monkeyBotDynamics.jl"))

monkeybot_dynamics (generic function with 1 method)

In [3]:
# function monkeybot_dynamics(params, x, τ, ct)
#     # Dynamics for monkeybot
#     # params = vect
    
#     # Unpack params for use in dynamics
#     g = params.g
#     l1, l2, L = params.l1, params.l2, params.L 
#     I1, I2, I = params.I1, params.I2, params.I
#     M, m1, m2 = params.M, params.m1, params.m2
#     a1, a2 = params.a1, params.a2

#     # Unpack the state vector and velocity vector
#     # Standard convenction -> x = [q, ̇q], ̇x = [̇q, ̈q]
#     θ1, θ2, px, py, θ = x[1:5]
#     dθ1, dθ2, dpx, dpy, dθ = x[6:10]
    
#     q = [θ1, θ2, px, py, θ]
#     dq = [dθ1, dθ2, dpx, dpy, dθ]
    
#     # Preallocate Space for xddot
#     xdot = zeros(10,1)

#     # Control
#     tau1, tau2 = τ[1:2]
    
#     #Arms in Contact represented with u1, u2
#     u1, u2 = ct[1:2]
    
#     #dx coeff
#     c1 = [m1*a1*l1*L*sin(θ1)*dθ 0 0 0 m1*a1*l1*L*sin(θ1)*dθ1
#         zeros(4,1) zeros(4,1) zeros(4,1) zeros(4,1) zeros(4,1)]
#     c2 = [0 0 0 0 0 
#             0 m2*a2*l2*L*sin(θ2)*dθ 0 0 m2*a2*l2*L*sin(θ2)*dθ
#             zeros(3,1) zeros(3,1) zeros(3,1) zeros(3,1) zeros(3,1)]
#     c3 = [-0.5*m1*L*a1*l1*sin(θ1)*dθ^2 0 0 0 -0.5*m1*L*a1*l1*sin(θ1)*(dθ+dθ1)
#             zeros(4,1) zeros(4,1) zeros(4,1) zeros(4,1) zeros(4,1)]
#     c4 = [0 0 0 0 0 
#             0 -0.5*m2*L*a2*l2*sin(θ2)*dθ^2 0 0 -0.5*m2*L*a2*l2*sin(θ2)*(dθ+dθ2)
#             zeros(3,1) zeros(3,1) zeros(3,1) zeros(3,1) zeros(3,1)]
#     coeff1 = c1+c2-c3-c4
# #     @show coeff1
# #     @show n, m = size(coeff1)
    
#     #ddx coeff
#     #3
#     c1 = [I1+m1*a1^2*l1^2 0 0 0 m1*a1^2*l1^2+m1*a1*l1*L*cos(θ1)
#             zeros(4,1) zeros(4,1) zeros(4,1) zeros(4,1) zeros(4,1)]
#     c2 = [0 0 0 0 0 
#             0 I2+m2*a2^2*l2^2 0 0 m2*a2^2*l2^2 + m2*a2*l2*L*cos(θ2)
#             zeros(3,1) zeros(3,1) zeros(3,1) zeros(3,1) zeros(3,1)]
#     c3 = zeros(5,5)
#     c3[3,3] = M+m1+m2
#     c3[4,4] = M+m1+m2
#     c3[5,5] = I+I1+I2
#     coeff2 = c1+c2 +c3
# #     @show coeff2
# #     @show n, m = size(coeff2)
    
#     #2 
#     dVdq = [m1*g*a1*l1*cos(θ+θ1)
#             m2*g*a2*l2*cos(θ+θ2)
#             0
#             M*g + m2*g + m1*g
#             m1*g*((2/L)*cos(θ) + a1*l1*cos(θ+θ1)) + m2*g*((2/L)*cos(θ) + a2*l2*cos(θ+θ2))]
    
# #     #Equation for A^T*λ
# #     A_t = [u1*l1*sin(θ+θ1) -u1*l1*cos(θ+θ1) 0 0
# #             0 0 u2*l2*sin(θ+θ2) -u2*l2*cos(θ+θ2)
# #             1*u1 0 1*u2 0
# #             0 1*u1 0 1*u2
# #             u1*((L/2)*sin(θ)+l1*sin(θ+θ1)) u1*((-L/2)*cos(θ)-l1*cos(θ+θ1)) u2*((L/2)*sin(θ)+l2*sin(θ+θ2)) u2*((-L/2)*cos(θ)-l2*cos(θ+θ2))]
    
#     function computeA(x)
#         θ1, θ2, px, py, θ = x
#         A = [u1*l1*sin(θ+θ1) 0                   1*u1     0       u1*((L/2)*sin(θ)+l1*sin(θ+θ1))
#             -u1*l1*cos(θ+θ1) 0                   0        1*u1    u1*((-L/2)*cos(θ)-l1*cos(θ+θ1))
#             0                u2*l2*sin(θ+θ2)     1*u2     0       u2*((L/2)*sin(θ)+l2*sin(θ+θ2))
#             0               -u2*l2*cos(θ+θ2)     0        1*u2    u2*((-L/2)*cos(θ)-l2*cos(θ+θ2))]
            
# #         A_t = [u1*l1*sin(θ+θ1) -u1*l1*cos(θ+θ1) 0 0;
# #             0 0 u2*l2*sin(θ+θ2) -u2*l2*cos(θ+θ2) 
# #             1*u1 0 1*u2 0 
# #             0 1*u1 0 1*u2 
# #             u1*((L/2)*sin(θ)+l1*sin(θ+θ1)) u1*((-L/2)*cos(θ)-l1*cos(θ+θ1)) u2*((L/2)*sin(θ)+l2*sin(θ+θ2)) u2*((-L/2)*cos(θ)-l2*cos(θ+θ2))]
#         return A
#     end
    
#     A = computeA(q)
#     A_t = transpose(A)

#     dvecA_dx = FD.jacobian(_q -> vec(computeA(_q)), q)
#     dvecA_dt = dvecA_dx*dq
#     dA = reshape(dvecA_dt, 4,5)
# #     @show A
# #     @show n, m = size(A)
# #     @show A_t
# #     @show dA
# #     @show n, m = size(dA)

# #     dA_t[1]  = FD.gradient(dθ1 -> A(dθ1, θ2, px, py, θ), θ1)
# #     dA_t[2]  = FD.gradient(dθ2 -> A(θ1, dθ2, px, py, θ), θ2)
# #     dA_t[3]  = FD.gradient(dpx -> A(θ1, θ2, dpx, py, θ), px)
# #     dA_t[4]  = FD.gradient(dpy -> A(θ1, θ2, px, dpy, θ), py)
# #     dA_t[5]  = FD.gradient(dθ -> A(d1, θ2, px, py, dθ), θ)
# #     ddq = inv(coeff2)*(u - coeff1*dx-dVdq + A_t*λ) # Only Works if coeff2 isnt singular
# #     @show coeff1
# #     @show [coeff1 A_t; A zeros(4,4)]
# #     @show n, m = size(dq)
    
#     out = [coeff1 A_t; A zeros(4,4)]\[τ - dVdq; zeros(4,1)] - [coeff1 A_t; A zeros(4,4)]\([coeff2; dA]*dq)
#     ddq = out[1:5]
#     xdot[1:5] = dq
#     @show 2
#     xdot[6:10] = ddq
#     @show 3
#     return xdot
# end

In [38]:
#Stance Dynamics
function stance1_dynamics(params::NamedTuple, x::Vector, u::Vector)
    #Dynamics with both hands constrained on the same bar
    c = [1,1]
    ẋ = monkeybot_dynamics(params, x, u, c)
    @show 1
    return ẋ
end

function stance2_dynamics(params::NamedTuple, x::Vector, u::Vector)
    #Dynamics with hand 1 constrained on a bar and hand 2 free floating
    c = [1,0]
    ẋ = monkeybot_dynamics(params, x, u, c)
    return ẋ
end

function stance3_dynamics(params::NamedTuple, x::Vector, u::Vector)
    #Dynamics with hand 1 on one bar and hand 2 on a different bar
    c = [1,1]
    ẋ = monkeybot_dynamics(params, x, u, c)
    return ẋ
end

function stance4_dynamics(params::NamedTuple, x::Vector, u::Vector)
    #Dynamics with hand 2 constrained on a bar and hand 1 free floating
    c = [0,1]
    ẋ = monkeybot_dynamics(params, x, u, c)
    return ẋ
end

#Jump maps
function jump1_map(x)
    # hand 1 experiences inelastic collision with bar
    
    #Zero out hand 1 linear velocity
    
    return xn
end

function jump2_map(x)
    # hand 2 experiences inelastic collision with bar
    
    #Zero out hand 2 linear velocity
    
    return xn
end

function jump3_map(x)
    # hands 1 & 2 experience inelastic collision with bar
    
    #Zero out both hand linear velocities
    
    return xn
end

#Generate Integrator Function
function dynamics_rk4(model::NamedTuple, ode::Function, x::Vector, u::Vector, dt::Real)::Vector
    k1 = dt * ode(model, x,        u)
    k2 = dt * ode(model, x + k1/2, u)
    k3 = dt * ode(model, x + k2/2, u)
    k4 = dt * ode(model, x + k3,   u)
    return x + (1/6)*(k1 + 2*k2 + 2*k3 + k4)
end


dynamics_rk4 (generic function with 1 method)

In [22]:

function getContactModeSequence(N::Int, B1::Vector, B2::Vector)
    
    
    return M1, M2, M3, M4
end


getContactModeSequence (generic function with 1 method)

In [23]:
#Cost Functions
function cost(params::NamedTuple, Z::Vector)::Real
    #Calculate cost over whole trajectory
    N, idx = params.N, params.idx
    Q, R, Qf = params.Q, params.R, params.Qf
    
    cost = 0.0
    for i = 1:(N-1)
        ##Maybe try using Xref for shoulder trajectory as cost to make sure it stays on that path
            ## Might interfere with free swing at maximum swing
        xi = Z[idx.x[i]]
        ui = Z[idx.u[i]]
        cost += 0.5*(xi'*Q*xi + ui'*R*ui)
    end
    xN = Z[idx.x[N]]
    cost += 0.5*(xN'*Qf*xN)
    
    return cost
end

function cost_minTime(params::NamedTuple, Z::Vector)::Real
    #Calculate cost over whole trajectory
    N, idx = params.N, params.idx
    Q, R, Qf = params.Q, params.R, params.Qf
    
    cost = 0.0
    for i = 1:(N-1)
        ##Maybe try using Xref for shoulder trajectory as cost to make sure it stays on that path
            ## Might interfere with free swing at maximum swing
        xi = Z[idx.x[i]]
        ui = Z[idx.u[i]]
        hi = Z[idx.h[i]] #Scale by time step for minimum time cost function
        cost += 0.5*hi*(xi'*Q*xi + ui'*R*ui)
    end
    xN = Z[idx.x[N]]
    cost += 0.5*(xN'*Qf*xN)
    
    return cost
end

cost_minTime (generic function with 1 method)

In [24]:
#Generate initial trajectory
function generateInitialTrajectory(params::NamedTuple, Z::Vector)
end

function generateSimpleTrajectory(params::NamedTuple, Z::Vector)
    #Generate initial simplified version of center body trajectory (sinusoidal path)
    
end

initialTrajectory = generateInitialTrajectory(params, x, u)
plot(initialTrajectory)

LoadError: UndefVarError: x not defined

In [36]:
function create_idx(nx,nu,N) #copied from homeworks
    # create idx for indexing convenience
    # x_i = Z[idx.x[i]]
    # u_i = Z[idx.u[i]]
    # h_i = Z[idx.h[i]]
    # and stacked dynamics constraints of size nx are 
    # c[idx.c[i]] = <dynamics constraint at time step i>
    #
    # feel free to use/not use this 
    
    # our Z vector is [x0, u0, x1, u1, …, xN]
    nz = (N-1) * nu + N * nx # length of Z 
    x = [(i - 1) * (nx + nu) .+ (1 : nx) for i = 1:N]
    u = [(i - 1) * (nx + nu) .+ ((nx + 1):(nx + nu)) for i = 1:(N - 1)]
#     h = [(i - 1) * (nx + nu + 1) .+ (nx + nu + 1) for i = 1:(N - 1)]
    
    # constraint indexing for the (N-1) dynamics constraints when stacked up
    c = [(i - 1) * (nx) .+ (1 : nx) for i = 1:(N - 1)]
    nc = (N - 1) * nx # (N-1)*nx 
    
#     return (nx=nx, nu=nu, N=N, nz=nz, nc=nc, x=x, u=u, h=h, c = c)
    return (nx=nx, nu=nu, N=N, nz=nz, nc=nc, x=x, u=u, c = c)
end

#Constraint Functions
function dynamics_constraints(params::NamedTuple, Z::Vector)::Vector
    idx, N, dt = params.idx, params.N, params.dt
    M1, M2, M3, M4 = params.M1, params.M2, params.M3, params.M4
    J1, J2 = params.J1, params.J2 
    @show 1
    
    # create c in a ForwardDiff friendly way (check HW0)
    c = zeros(eltype(Z), idx.nc)
    
    # TODO: input walker dynamics constraints (constraints 3-6 in the opti problem)
    for k = 1:(N-1)
        xk = Z[idx.x[k]]
        uk = Z[idx.u[k]]
        if (k in M1) #&& !(k in J1) && !(k in J2)
            @show size(xk)
            @show size(uk)
            @show size(Z[idx.x[k+1]])
            c[idx.c[k]] = Z[idx.x[k+1]] - dynamics_rk4(params, stance1_dynamics, xk, uk, dt)
            @show 3
        elseif (k in M2) #&& !(k in J1)
            c[idx.c[k]] = Z[idx.x[k+1]] - dynamics_rk4(params, stance2_dynamics, xk, uk, dt)
            
        elseif (k in M3) #&& !(k in J1) && !(k in J2)
            c[idx.c[k]] = Z[idx.x[k+1]] - dynamics_rk4(params, stance3_dynamics, xk, uk, dt)
            
        elseif (k in M4) #&& !(k in J1)
            c[idx.c[k]] = Z[idx.x[k+1]] - dynamics_rk4(params, stance4_dynamics, xk, uk, dt)
            
#         elseif (k in M2) && (k in J2) #hand 2 grabs bar
#             c[idx.c[k]] = Z[idx.x[k+1]] - jump2_map(dynamics_rk4(params, stance2_dynamics, xk, uk, dt))
        
#         elseif (k in M4) && (k in J1) #hand 1 grabs bar
#             c[idx.c[k]] = Z[idx.x[k+1]] - jump1_map(dynamics_rk4(params, stance4_dynamics, xk, uk, dt))
                
        end
    end
    
    return c 
end

function dynamics_constraints_minTime(params::NamedTuple, Z::Vector)::Vector
    idx, N = params.idx, params.N
    M1, M2, M3, M4 = params.M1, params.M2, params.M3, params.M4
    J1, J2 = params.J1, params.J2 

    # create c in a ForwardDiff friendly way (check HW0)
    c = zeros(eltype(Z), idx.nc)
    
    # TODO: input walker dynamics constraints (constraints 3-6 in the opti problem)
    for k = 1:(N-1)
        xk = Z[idx.x[k]]
        uk = Z[idx.u[k]]
        hk = Z[idx.h[k]]
        if (k in M1) && !(k in J1) && !(k in J2)
            c[idx.c[k]] = Z[idx.x[k+1]] - dynamics_rk4(params, stance1_dynamics, xk, uk, hk)
            
        elseif (k in M2) && !(k in J1)
            c[idx.c[k]] = Z[idx.x[k+1]] - dynamics_rk4(params, stance2_dynamics, xk, uk, hk)
            
        elseif (k in M3) && !(k in J1) && !(k in J2)
            c[idx.c[k]] = Z[idx.x[k+1]] - dynamics_rk4(params, stance3_dynamics, xk, uk, hk)
            
        elseif (k in M4) && !(k in J1)
            c[idx.c[k]] = Z[idx.x[k+1]] - dynamics_rk4(params, stance4_dynamics, xk, uk, hk)
            
        elseif (k in M2) && (k in J2) #hand 2 grabs bar
            c[idx.c[k]] = Z[idx.x[k+1]] - jump2_map(dynamics_rk4(params, stance2_dynamics, xk, uk, hk))
        
        elseif (k in M4) && (k in J1) #hand 1 grabs bar
            c[idx.c[k]] = Z[idx.x[k+1]] - jump1_map(dynamics_rk4(params, stance4_dynamics, xk, uk, hk))
                
        end
    end
    
    return c 
end

function bar_constraint(params::NamedTuple, Z::Vector)::Vector
    #Index to correct bar depending on expected contact mode and check to make sure contraint is met
    #unpack params for use in dynamics
    g = params.g
    N = params.N
    l1, l2, L = params.l1, params.l2, params.L 
    I1, I2, I = params.I1, params.I2, params.I
    M, m1, m2 = params.M, params.m1, params.m2
    @show 3
    #Add constraint for odd bars on arm 1 and even bars on arm 2
    c = zeros(eltype(Z), 4*N)
    for k = 1:N
        xk = Z[idx.x[k]]
        th1k, th2k, Pxk, Pyk, thk = xk
        
        #Add constraints to hands as necessary
        if (k in M1 || k in M3)
            #Get current bar coordinates
            x1g = BarCoords[B1[i]]
            x2g = BarCoords[B2[i]]
            
            #Hand 1 constraints
            c[4*k] = Pxyk - cos(thk)*L/2 - cos(thk + th1k)*l1 - x1g[1]
            c[4*k+1] = Pxyk + sin(thk)*L/2 + sin(thk + th1k)*l1 - x1g[2]
            
            #Hand 2 constraints
            c[4*k+2] = Pxyk -cos(thk)*L/2 - cos(thk + th2k)*l2 - x2g[1]
            c[4*k+3] = Pxyk + sin(thk)*L/2 + sin(thk + th2k)*l2 - x2g[2]
            
        elseif (k in M2)
            #Get current bar coordinates
            x1g = BarCoords[B1[i]]

            #Hand 1 constraints
            c[2*k] = Pxyk - cos(thk)*L/2 - cos(thk + th1k)*l1 - x1g[1]
            c[2*k+1] = Pxyk + sin(thk)*L/2 + sin(thk + th1k)*l1 - x1g[2]
            
        elseif (k in M4)
            #Get current bar coordinates
            x2g = BarCoords[B2[i]]
            
            #Hand 2 constraints
            c[2*k] = Pxyk - cos(thk)*L/2 - cos(thk + th2k)*l2 - x2g[1]
            c[2*k+1] = Pxyk + sin(thk)*L/2 + sin(thk + th2k)*l2 - x2g[2]
            
        end
    end
    
    return c
end

function equality_constraint(params::NamedTuple, Z::Vector)::Vector
    N, idx, xic, xg = params.N, params.idx, params.xic, params.xg 
    
    #Stack up all equality constraints
    initialCondition = Z[idx.x[1]] - xic
    terminalCondition = Z[idx.x[N]] - xg
    dynamicsConstraints = dynamics_constraints(params, Z)
    stanceConstraints = stance_constraint(params, Z)
    
    return [initialCondition; terminalCondition; dynamicsConstraints; stanceConstraints]
end

function equality_constraint_minTime(params::NamedTuple, Z::Vector)::Vector
    N, idx, xic, xg = params.N, params.idx, params.xic, params.xg 
    
    #Stack up all equality constraints
    initialCondition = Z[idx.x[1]] - xic
    terminalCondition = Z[idx.x[N]] - xg
    dynamicsConstraints = dynamics_constraints_minTime(params, Z)
    stanceConstraints = bar_constraint(params, Z)
    
    return [initialCondition; terminalCondition; dynamicsConstraints; stanceConstraints]
end

function inequality_constraint(params::NamedTuple, Z::Vector)::Vector
    idx, N, dt = params.idx, params.N, params.dt
    M1, M2 = params.M1, params.M2 
        
    #Write inequality constraints for the problem
        
    return c
end


inequality_constraint (generic function with 1 method)

In [61]:
# problem size 
nx = 10
nu = 2
nu_minTime = 3
tf = 10
dt = 0.1
t_vec = 0:dt:tf
N = length(t_vec)
Nbars = 5
g = 9.81
l1 = 1
l2 =1
L = 1
I1 = 0.2
I2 = 0.2
I = 0.2
M = 1
m1 = 1
m2 = 1
a1 = 0.5
a2 = 0.5

BarCoords = [zeros(2) for i = 1:Nbars]
for i = 1:Nbars
    barX = (i-1)*2
    barY = 0
    BarCoords[i] = [barX; barY]
end
@show BarCoords

#Inital and final conditions
θ1, θ2, θ = 0, 0, pi/3
dθ1, dθ2, dθ = 0.1, 0.1, 0.1
px, py = 0,-l1
dpx, dpy = 0,0
xic = [θ1, θ2, px, py, θ, dθ1, dθ2, dpx, dpy, dθ]
xg = [0,pi/3,BarCoords[Nbars][1],(BarCoords[Nbars][1]-l1),pi/2,0,0,0,0,0]

# index sets 
B_1 = [1,1,1,0,3,3,3,0,5,5]
B_2 = [1,0,2,2,2,0,4,4,4,0]
B1 = vcat([B_1[j] for j = 1:(length(B_1)) for i = 1:(N/length(B_1))]) #Bar indecies for hand 1
B2 = vcat([B_2[j] for j = 1:(length(B_2)) for i = 1:(N/length(B_2))]) #Bar indecies for hand 2
M1 = zeros(Int, 1, 0)
M2 = zeros(Int, 1, 0)
M3 = zeros(Int, 1, 0)
M4 = zeros(Int, 1, 0)
J1 = zeros(Int, 1, 0)
J2 = zeros(Int, 1, 0)
previouslyM2 = false
previouslyM4 = false
for i = 1:length(B1)
    if (B1[i] == 1 && B2[i] == 1)
        M1 = vcat([M1 i])
    elseif (B1[i] >= 1 && B2[i] == 0)
        M2 = vcat([M2 i])
        previouslyM2 = true
    elseif (B1[i] >= 1 && B2[i] > 1)
        M3 = vcat([M3 i])
        if previouslyM2
            J1 = vcat([J1 i-1])
            previouslyM2 = false
        elseif previouslyM4
            J2 = vcat([J2 i-1])
            previouslyM4 = false
        end
    elseif (B1[i] == 0 && B2[i] > 1)
        M4 = vcat([M4 i])
        previouslyM4 = true
    end
end
@show M1
@show M2
@show M3
@show M4
@show J1
@show J2
@show B1
@show B2

# # Create reference trajectory for shoulder joint
# Xref, Uref = shoulder_reference_trajectory(params, xic, xg, dt, N)

# LQR cost function
######CHANGE########
Q = diagm([1; 10; fill(1.0, 4); 1; 10; fill(1.0, 4)]);
R = diagm([10; 10])
Qf = 1*Q;

# create indexing utilities 
idx = create_idx(nx,nu,N)
@show N
# Params
params = (
    nx = nx,
    nu = nu,
    tf = tf, 
    dt = dt, 
    t_vec = t_vec,
    N = N, 
    M1 = M1, 
    M2 = M2,
    M3 = M3,
    M4 = M4,
    J1 = J1, 
    J2 = J2,
    B1 = B1, 
    B2 = B2,
    xic = xic, 
    xg = xg,
    idx = idx,
    Q = Q, R = R, Qf = Qf,
#     Xref = Xref, 
#     Uref = Uref,
    g = g,
    l1 = l1, l2 = l2, L = L,
    I1 = I1, I2 = I2, I = I,
    M = M, m1 = m1, m2 = m2,
    a1 = a1, a2 = a2
)

#Primal bounds
x_l = -Inf*ones(idx.nz) # update this 
x_u =  Inf*ones(idx.nz) # update this
for i = 1:N
    xi_l = -Inf*ones(idx.nx)
    xi_l[5] = 0
    xi_u = Inf*ones(idx.nx)
    xi_u[5] = pi
    x_l[idx.x[i]] = xi_l
    x_u[idx.x[i]] = xi_u
#     #Add in min time constraints
#     x_l[idx.h[i]] = h_min
#     x_u[idx.h[i]] = h_max
end

#Inequality constraint bounds
c_l = 0.5*ones(2*N) # update this 
c_u = 1.5*ones(2*N) # update this 

#Initialize z0 with reference shoulder trajectory
z0 = zeros(idx.nz) # update this
# for i = 1:(N-1)
#     z0[idx.x[i]] = Xref[i]
#     z0[idx.u[i]] = Uref[i]
# end
# z0[idx.x[N]] = Xref[N]

# adding a little noise to the initial guess is a good idea 
z0 = z0 + (1e-6)*randn(idx.nz)

#Choose diff type
diff_type = :auto 

#Solve trajectory
Z = fmincon(cost,equality_constraint,inequality_constraint,
            x_l,x_u,c_l,c_u,z0,params,diff_type;
            tol = 1e-6, c_tol = 1e-6, max_iters = 10_000, verbose = true)

# pull the X and U solutions out of Z 
X = [Z[idx.x[i]] for i = 1:N]
U = [Z[idx.u[i]] for i = 1:(N-1)]
dt = [Z[idx.h[i]] for i = 1:(N-1)]

#Plot data
Xm = hcat(X...)
Um = hcat(U...)
dt = hcat(dt...)

#Plot joint angles
plot(dt[1,:],Xm[1,:], label = "Theta 1")
plot!(dt[1,:],Xm[2,:], label = "Theta 2")
display(plot!(dt[1,:],Xm[5,:], label = "Theta Body", xlabel = "Time (s)",
              ylabel = "Angle (rad)", title = "Joint Angles v.s. Time"))

#Plot shoulder X,Y
display(plot!(Xm[2,:],Xm[3,:], label = "Shoulder Position", xlabel = "x (m)",
              ylabel = "y (m)", title = "Soulder Position"))


BarCoords = [[0.0, 0.0], [2.0, 0.0], [4.0, 0.0], [6.0, 0.0], [8.0, 0.0]]
M1 = [1 2 3 4 5 6 7 8 9 10]
M2 = [11 12 13 14 15 16 17 18 19 20 51 52 53 54 55 56 57 58 59 60 91 92 93 94 95 96 97 98 99 100]
M3 = [21 22 23 24 25 26 27 28 29 30 41 42 43 44 45 46 47 48 49 50 61 62 63 64 65 66 67 68 69 70 81 82 83 84 85 86 87 88 89 90]
M4 = [31 32 33 34 35 36 37 38 39 40 71 72 73 74 75 76 77 78 79 80]
J1 = [20 60]
J2 = [40 80]
B1 = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]
B2 = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,

LoadError: UndefVarError: coeff1 not defined